# Menghitung Jarak (Similarity)

**Nama  : Rizki Ardian Samudra**

**NIM   : 210411100179**

**Kelas : Penambangan Data-C**

1.   Code untuk membaca file csv dari url drive dan menampikan tabel datanya

In [ ]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import pairwise_distances
import pandas as pd
import numpy as np

file_url = 'https://drive.google.com/file/d/1Ff6Vj1jxHa_GULkDUQ0sI0aaSRxPfhiW/view?usp=sharing'

file_id = file_url.split('/')[-2]
dwn_url = "https://drive.google.com/uc?id=" + file_id
df = pd.read_csv(dwn_url, sep=';')
df = df.drop(columns=['Unnamed: 10', 'Unnamed: 11', 'No'])
df.head()

,Nama,Umur,Tinggi,Jenis Kelamin,Status,Pendapatan,Keahlian,Pekerjaan,Hobi
0,Rizki,20,170,Laki-laki,Belum Menikah,tinggi,mahir,guru,renang
1,Ardi,30,160,Laki-laki,Menikah,menengah,menengah,dokter,ngoding
2,Sania,25,172,Perempuan,Belum Menikah,rendah,pemula,sopir,desain
3,Nugroho,45,181,Laki-laki,Menikah,tinggi,mahir,pilot,ngopi
4,Bella,34,155,Perempuan,Belum Menikah,tinggi,mahir,programmer,futsal


>> Penjelasan Tipe Data :

>    *   Kolom "Umur" dan "Tinggi" adalah tipe data numerik karena nilainya berupa angka.
>    *   Kolom "Jenis Kelamin" dan "Status" adalah tipe data biner simetris karena nilainya hanya berupa dua opsi.
>    *   Kolom "Pendapatan" dan "Keahlian" adalah tipe data ordinal karena nilainya memiliki arti urutan atau peringkat di antara nilai-nilai yang ada.
>    *   Kolom "Pekerjaan" dan "Hobi" adalah tipe data categorical karena nilainya mewakili beberapa jenis kategori dan nilai-nilainya tidak memiliki tingkatan.

2. Normalisasi

In [ ]:
# Copy Data Frame
data = df.copy()

# Mengambil kolom-kolom yang diinginkan
selected_columns = ['Umur', 'Tinggi', 'Jenis Kelamin', 'Status', 'Pendapatan', 'Keahlian', 'Pekerjaan', 'Hobi']
selected_data = data[selected_columns]

# Kolom numerik
numeric_columns = ['Umur', 'Tinggi']
numeric_data = selected_data[numeric_columns]
# Melakukan normalisasi Min-Max Scaling pada kolom numerik
scaler = MinMaxScaler()
normalized_numeric_data = scaler.fit_transform(numeric_data)

# Kolom biner simetris
binary_columns = ['Jenis Kelamin', 'Status']
binary_data = selected_data[binary_columns]
# Mengubah kolom biner simetris menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
encoded_binary_data = binary_data.apply(label_encoder.fit_transform)

# Kolom ordinal
ordinal_columns = ['Pendapatan', 'Keahlian']
ordinal_data = selected_data[ordinal_columns]
# Mengubah kolom ordinal menjadi angka menggunakan OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories=[['rendah', 'menengah', 'tinggi'], ['pemula', 'menengah', 'mahir']])
encoded_ordinal_data = ordinal_encoder.fit_transform(ordinal_data)


# Kolom kategori
categorical_columns = ['Pekerjaan', 'Hobi']
categorical_data = selected_data[categorical_columns]
# Mengubah kolom kategori menjadi one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_categorical_data = onehot_encoder.fit_transform(categorical_data)


# Menggabungkan kembali data yang telah dinormalisasi dan dikodekan
normalized_data = pd.DataFrame(
    data={
        'Umur': normalized_numeric_data[:, 0],
        'Tinggi': normalized_numeric_data[:, 1],
        'Jenis Kelamin': encoded_binary_data['Jenis Kelamin'],
        'Status': encoded_binary_data['Status'],
        'Pendapatan': encoded_ordinal_data[:, 0],
        'Keahlian': encoded_ordinal_data[:, 1]
    }
)

# Membuat nama fitur untuk kolom kategori
onehot_columns = []
for col, categories in zip(categorical_columns, onehot_encoder.categories_):
    for category in categories:
        onehot_columns.append(f'{col}_{category}')

normalized_data[onehot_columns] = encoded_categorical_data

print("Data setelah normalisasi:")
normalized_data.head()

Data setelah normalisasi:


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Umur,Tinggi,Jenis Kelamin,Status,Pendapatan,Keahlian,Pekerjaan_dokter,Pekerjaan_guru,Pekerjaan_pilot,Pekerjaan_programmer,Pekerjaan_sopir,Hobi_desain,Hobi_futsal,Hobi_ngoding,Hobi_ngopi,Hobi_renang
0,0.00,0.576923,0,0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.40,0.192308,0,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.20,0.653846,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1.00,1.000000,0,1,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.56,0.000000,1,0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


3. Hasil Jarak Manhattan

In [ ]:
# Menghitung jarak Manhattan antara setiap pasangan titik dalam dataset yang telah dinormalisasi
distances = pairwise_distances(normalized_data, metric='manhattan')

print("Matrix Jarak Manhattan (setelah normalisasi):")
print(distances)

Matrix Jarak Manhattan (setelah normalisasi):
[[ 0.          7.78461538  9.27692308  6.42307692  6.13692308]
 [ 7.78461538  0.          8.66153846  7.40769231  8.35230769]
 [ 9.27692308  8.66153846  0.         11.14615385  9.01384615]
 [ 6.42307692  7.40769231 11.14615385  0.          7.44      ]
 [ 6.13692308  8.35230769  9.01384615  7.44        0.        ]]


coba baris kesatu

In [ ]:
# Mengurutkan indeks jarak terkecil dari baris pertama
sorted_indices = np.argsort(distances[0])

# Mengambil dua indeks dengan jarak terdekat
closest_indices = sorted_indices[:3]

# Menampilkan data dengan jarak terdekat
for index in closest_indices:
    closest_data = selected_data.iloc[index]
    print("Data dengan jarak terdekat:")
    print(closest_data)
    print("Jarak: ", distances[0][index])
    print()

Data dengan jarak terdekat:
Umur                        20
Tinggi                     170
Jenis Kelamin        Laki-laki
Status           Belum Menikah
Pendapatan              tinggi
Keahlian                 mahir
Pekerjaan                 guru
Hobi                    renang
Name: 0, dtype: object
Jarak:  0.0

Data dengan jarak terdekat:
Umur                        34
Tinggi                     155
Jenis Kelamin        Perempuan
Status           Belum Menikah
Pendapatan              tinggi
Keahlian                 mahir
Pekerjaan           programmer
Hobi                    futsal
Name: 4, dtype: object
Jarak:  6.136923076923077

Data dengan jarak terdekat:
Umur                    45
Tinggi                 181
Jenis Kelamin    Laki-laki
Status             Menikah
Pendapatan          tinggi
Keahlian             mahir
Pekerjaan            pilot
Hobi                 ngopi
Name: 3, dtype: object
Jarak:  6.4230769230769225



coba baris kedua

In [ ]:
# Mengurutkan indeks jarak terkecil dari baris kedua
sorted_indices = np.argsort(distances[1])

# Mengambil dua indeks dengan jarak terdekat
closest_indices = sorted_indices[:3]

# Menampilkan data dengan jarak terdekat
for index in closest_indices:
    closest_data = selected_data.iloc[index]
    print("Data dengan jarak terdekat:")
    print(closest_data)
    print("Jarak: ", distances[1][index])
    print()

Data dengan jarak terdekat:
Umur                    30
Tinggi                 160
Jenis Kelamin    Laki-laki
Status             Menikah
Pendapatan        menengah
Keahlian          menengah
Pekerjaan           dokter
Hobi               ngoding
Name: 1, dtype: object
Jarak:  0.0

Data dengan jarak terdekat:
Umur                    45
Tinggi                 181
Jenis Kelamin    Laki-laki
Status             Menikah
Pendapatan          tinggi
Keahlian             mahir
Pekerjaan            pilot
Hobi                 ngopi
Name: 3, dtype: object
Jarak:  7.407692307692308

Data dengan jarak terdekat:
Umur                        20
Tinggi                     170
Jenis Kelamin        Laki-laki
Status           Belum Menikah
Pendapatan              tinggi
Keahlian                 mahir
Pekerjaan                 guru
Hobi                    renang
Name: 0, dtype: object
Jarak:  7.784615384615385

